In [24]:
import os
import logging
from langchain.text_splitter import MarkdownTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import NotebookLoader
import json
from dotenv import load_dotenv

load_dotenv()
logging.basicConfig(level=logging.INFO)

In [ ]:
base_paths = ["3. LangChain/", "4. RAG/"]

documents = []

# Step 1: .ipynb 파일 불러오기
for base_path in base_paths:
    notebooks = [os.path.join(base_path, f) for f in os.listdir(base_path) if f.endswith('.ipynb')]
    
    # Step 2: 각 .ipynb 파일을 읽고 문서로 변환
    for file in notebooks:
        loader = NotebookLoader(path=file)
        loaded_docs = loader.load()  
        
        for doc in loaded_docs:
            doc.metadata['source_file'] = os.path.basename(file) 
            doc.metadata['source_folder'] = os.path.basename(os.path.normpath(base_path))  
        documents.extend(loaded_docs) 

# Step 3: 텍스트 분할 및 임베딩
text_splitter = MarkdownTextSplitter(
    chunk_size=600,   
    chunk_overlap=200  
)

docs = []
for doc in documents:
    split_docs = text_splitter.create_documents([doc.page_content])
    for split_doc in split_docs:
        split_doc.metadata = doc.metadata
    docs.extend(split_docs)

for i, doc in enumerate(docs[:5]):
    print(f"[Chunk {i}]\n")
    print("Content:")
    print(doc.page_content)
    print("\nMetadata:")
    print(doc.metadata) 
    print("=" * 60)

[Chunk 0]

Content:
'markdown' cell: '['# [LangChain](https://python.langchain.com/v0.2/docs/introduction/)']'

'markdown' cell: '['- LangChain은 대규모 언어 모델(LLM)을 기반으로 애플리케이션을 구축하기 위한 오픈 소스 프레임워크입니다.\n', '- LLM은 대량의 데이터로 사전 훈련된 대규모 딥 러닝 모델로서, 질문에 답하거나 텍스트 기반 프롬프트에서 이미지를 생성하는 등 사용자 쿼리에 대한 응답을 생성할 수 있습니다.\n', '- LangChain은 모델이 생성하는 정보의 맞춤화, 정확성 및 관련성을 개선하기 위한 도구와 추상화 기능을 제공합니다.\n', '  - 예를 들어 개발자는 LangChain 구성 요소를 사용하여 새 프롬프트 체인을 구축하거나 기존 템플릿을 맞춤화할 수 있습니다.\n', '  - 또한 LangChain에는 LLM이 재훈련 없이 새 데이터 세트에 액세스할 수 있도록 하는 구성 요소도 포함되어 있습니다.']'

Metadata:
{'source': 'db\\raw_data\\3. LangChain\\1. LangChain - 개요.ipynb', 'source_file': '1. LangChain - 개요.ipynb', 'source_folder': '3. LangChain'}
[Chunk 1]

Content:
'markdown' cell: '['## [LangChain 응용](https://velog.io/@kofsitho/Langchain-%ED%83%90%EC%83%89-%EA%B3%A0%EA%B8%89-%ED%94%84%EB%A0%88%EC%9E%84%EC%9B%8C%ED%81%AC%EB%A5%BC-%ED%86%B5%ED%95%9C-AI-%ED%98%81%EC%8B%A0)']'

'markdown' cell: '['- 인공지능(AI)이 산업과 일상생활을 빠르게 변화시키고 있는 세상에서 기술 발전의 선두에 선다는 것

In [ ]:
# Step 4: 임베딩 함수 초기화 및 FAISS 벡터 스토어 저장
try:
    logging.info("Initializing embedding function")
    embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")

    logging.info("Adding documents to FAISS vector store")
    vectorstore = FAISS.from_documents(
        documents=docs,
        embedding=embedding_function
    )
    logging.info("Documents added to FAISS vector store")

    vectorstore.save_local('./db/faiss')
    logging.info("FAISS vector store saved locally to './db/faiss'")

    metadata = [doc.metadata for doc in docs]
    with open('./db/faiss/metadata.json', 'w', encoding='utf-8') as f:
        json.dump(metadata, f, ensure_ascii=False, indent=4)
    logging.info("Metadata saved locally to './db/faiss/metadata.json'")

except Exception as e:
    logging.error(f"Error adding documents to FAISS vector store: {e}")
    exit(1)

INFO:root:Initializing embedding function
INFO:root:Adding documents to FAISS vector store
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /embeddings in 15.439000 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /embeddings in 14.483000 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Ret

In [ ]:

# Step 5 : Eval Test
embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.load_local('./db/faiss', embedding_function, allow_dangerous_deserialization=True)
logging.info("FAISS vector store loaded from './db/faiss'")

def retrieve_similar_documents(user_input):
    try:
        logging.info(f"Starting query search: {user_input}")

        retriever = vectorstore.as_retriever(search_type="similarity")
        search_results = retriever.invoke(user_input)

        logging.info("Query search completed")

        cleaned_results = []
        for doc in search_results:
            content = doc.page_content.strip()
            metadata = doc.metadata 
            cleaned_results.append({
                "content": content,
                "metadata": metadata 
            })
        return cleaned_results

    except Exception as e:
        logging.error(f"Error during query search: {e}")
        return []

INFO:root:FAISS vector store loaded from './db/faiss'


In [ ]:
retrieve_similar_documents('FAISS가 뭘까?')

INFO:root:Starting query search: FAISS가 뭘까?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:root:Query search completed


[{'content': "'markdown' cell: '['## FAISS (Facebook AI Similarity Search)']'\n\n'markdown' cell: '['- Faiss의 주요 개발은 Meta의 Fundamental AI Research 그룹에서 수행합니다.\\n', '- Faiss는 유사점을 신속하게 검색하고 밀집된 벡터의 클러스터링을 위한 오픈 소스 라이브러리입니다.\\n', '- 여기에는 RAM 용량을 초과할 수 있는 벡터 세트까지 다양한 크기의 벡터 세트 내에서 검색할 수 있는 알고리즘이 포함되어 있습니다.\\n', '- 또한 Faiss는 평가 및 매개변수 조정을 위한 보조 코드를 제공합니다.\\n', '\\n', '주로 C++로 코딩되었지만 Python/NumPy 통합을 완벽하게 지원합니다. 주요 알고리즘 중 일부는 GPU 실행에도 사용할 수 있습니다.']'",
  'metadata': {'source': 'db\\raw_data\\4. RAG\\5-1. Vector db - 개요.ipynb',
   'source_file': '5-1. Vector db - 개요.ipynb',
   'source_folder': '4. RAG'}},
 {'content': "'markdown' cell: '['## FAISS\\n', '- FAISS(Facebook AI Similarity Search)는 Facebook AI Research에 의해 개발된 라이브러리로, 대규모 벡터 데이터셋에서 유사도 검색을 빠르고 효율적으로 수행할 수 있게 해줍니다.\\n', '- FAISS는 특히 벡터의 압축된 표현을 사용하여 메모리 사용량을 최소화하면서도 검색 속도를 극대화하는 특징이 있습니다.']'",
  'metadata': {'source': 'db\\raw_data\\4. RAG\\5-3. RAG - FAISS.ipynb',
   'source_file': '5-3. RAG - FAISS.ipynb',
   'source_folder': '4. 